In [1]:
import numpy as np
import pandas as pd

import os
import cPickle as pickle

In [2]:
print ('Reading the orders dataset...')
df_orders = pd.read_csv('data/orders.csv')
print('Reading the train products dataset...')
df_products_train = pd.read_csv('data/order_products__train.csv')
print('Reading the prior products dataset...')
df_products_prior = pd.read_csv('data/order_products__prior.csv')

Reading the orders dataset...
Reading the train products dataset...
Reading the prior products dataset...


In [3]:
 def save_products(filename_train, filename_test,force=False):
    path_train = 'data/' + filename_train
    path_test = 'data/' + filename_test
    if(os.path.exists(path_train) and os.path.exists(path_test)  and not force) :
        
        print('Files {} and {} already exist.'.format(path_train,path_test))
        
    else:
        print('Reading the prior products dataset...')
        df_products_prior = pd.read_csv('data/order_products__prior.csv')
    
        users_train = df_orders.loc[df_orders['eval_set']=='train','user_id']
        df_orders_prior_train = df_orders.loc[df_orders['user_id'].isin(users_train)]
        df_orders_prior_train = df_orders_prior_train.loc[df_orders_prior_train['eval_set']=='prior']

        users_test  = df_orders.loc[df_orders['eval_set']=='test','user_id']
        df_orders_prior_test = df_orders.loc[df_orders['user_id'].isin(users_test)]
        df_orders_prior_test = df_orders_prior_test.loc[df_orders_prior_test['eval_set']=='prior']

        df_products_prior_train = df_products_prior.loc[df_products_prior['order_id'].isin(df_orders_prior_train['order_id'])]
        df_products_prior_test = df_products_prior.loc[df_products_prior['order_id'].isin(df_orders_prior_test['order_id'])]

        with open(path_train,'wb') as f:
            pickle.dump(df_products_prior_train,f, pickle.HIGHEST_PROTOCOL)
        with open(path_test,'wb') as f:
            pickle.dump(df_products_prior_test,f, pickle.HIGHEST_PROTOCOL)    

        del df_products_prior

save_products('order_products__prior_train.pickle',
              'order_products__prior_test.pickle')

Files data/order_products__prior_train.pickle and data/order_products__prior_test.pickle already exist.


In [13]:
with open('data/order_products__prior_train.pickle','rb') as f:
            df_products_prior_train = pickle.load(f)
df_products_prior_train.head(40)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [19]:
number_prod_per_order = df_products_prior_train.groupby('order_id')['product_id'].count()
number_prod_per_order.rename('number_prod_per_order',inplace=True)
number_prod_per_order[2398795]

6

In [12]:
df_orders_prior = df_orders.loc[df_orders['eval_set']=='prior']
.join(number_prod_per_order,how='right')
df_orders_prior.head(40)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,number_prod_per_order
order_id,,,,,,,,
2,473747,1,prior,3,3,12,21.0,9
3,2254736,1,prior,4,4,7,29.0,8
4,431534,1,prior,5,4,15,28.0,13
5,3367565,1,prior,6,2,7,19.0,26
6,550135,1,prior,7,1,9,20.0,3
7,3108588,1,prior,8,1,14,14.0,2
8,2295261,1,prior,9,1,16,0.0,1
9,2550362,1,prior,10,4,8,30.0,15
10,1187899,1,train,11,4,8,14.0,15


In [32]:
df_products_prior.loc[df_products_prior['order_id']==2539329]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [27]:
df_number_of_product = df_products_prior.groupby('order_id')['product_id'].count()
df_number_of_product.rename('Number_of_product_per_order',inplace=True)
df_number_of_product = pd.DataFrame(df_number_of_product)

In [28]:
df_number_of_product = df_number_of_product.join(df_orders.set_index('order_id')['user_id'])
df_number_of_product.head()

,Number_of_product_per_order,user_id
order_id,,
2,9,202279
3,8,205970
4,13,178520
5,26,156122
6,3,22352


In [29]:
df_number_of_product=df_number_of_product.groupby('user_id')['Number_of_product_per_order'].agg([np.mean,np.std])
df_number_of_product.to_csv('data/number_of_products_per_user.csv')